In [113]:
from math import sqrt
from src.TSP_algEvolutivi.fcOptimisGA.Chromosome import Chromosome
def fcTotalCost(param,route):
    travel_cost=0
    for i in range(len(route)):
        origin = route[i]
        #in cazul in care a juns la finalul routei , revine la primul nod
        if i == len(route) - 1:
            dest = route[0]
        else:
            dest = route[i+1]
        travel_cost += param['edges'][origin][dest]
    return travel_cost


def fcEval(route, param):
    #functia de fitness calculeaza inversul costului total 
    #TSP ca problema de optimiziare a fct de fitness ( se cauta maximul )
    return 1/fcTotalCost(param,route)

In [114]:
def distEuclid(x,y):
    return sqrt(pow(x[0] - y[0],2) + pow(x[0] - y[0],2))
def readFile(filename):
    f = open(filename, "r")
    graph = {}

    edges = []
    lines = f.readlines()
    n = int(lines[0])
    graph['noCities'] = n
    for i in range(1, n + 1):
        costs = []
        for j in lines[i].split(","):
            costs.append(int(j))
        edges.append(costs)
    graph['edges'] = edges
    return graph
def readFile2(filename):
    f = open(filename, "r")
    lines = f.readlines()
    lines = lines[6:]
    n = len(lines)
    graph = {}
    edges = []
    graph['noCities'] = n
    for i in range(0, n):
        node = []
        srcCoord = [int(lines[i].split(" ")[1]), int(lines[i].split(" ")[2])]
        for j in range(0, n):
            if i!=j:
                destCoord = [int(lines[j].split(" ")[1]), int(lines[j].split(" ")[2])]
                dist=distEuclid(srcCoord, destCoord)
                node.append(dist)
            else:
                node.append(0)
        edges.append(node)
    graph['edges'] = edges
    return graph

import networkx as nx
# read the graph's details



**GA running**

Use the GA (with real encoding) for identify the optimal solution


In [115]:



class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self,network):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres, network)
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness > self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.binomial_crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            pos=self.__param['noDim']+1
            for i in range(self.__param['noDim']):
                if off.repres[i]==0:
                    pos=i
            off.repres[0],off.repres[pos]=off.repres[pos],off.repres[0]
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self,network):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres, network)
            worst = self.worstChromosome()
            if off.fitness < worst.fitness:
                off = worst
            newPop.append(off)
        self.__population = newPop
        self.evaluation(network)

In [116]:
MIN=1
MAX=6
#from BinChromosome import Chromosome
from random import seed, randint 
import os
class Solver(object):
    def __init__(self):
        self._filename=""
        self._popSize=0
        self._noGen=0
    def setSolverParam(self,filename,popSize,noGen):
        self._filename=filename
        self._popSize=popSize
        self._noGen=noGen
    def solveGA(self):
        #generate seed
        seed()
        #read net
        
        #graph=readFile('50p_easy_01_tsp.txt')
        #graph=readFile('50p_hard_01_tsp.txt')
        #graph=readFile('50p_medium_01_tsp.txt')
        #graph=readFile('100p_fricker26.txt')
        graph=readFile2('150p_eil51.txt')
        #graph=readFile('input.txt')
        
        
        # initialise de GA parameters
        gaParam = {'popSize' : self._popSize, 'noGen' :self._noGen, 'pc' : 0.8, 'pm' : 0.5}
        # problem parameters
        problParam = {'cr' : 0.5, 'min' : MIN, 'max' : MAX, 'function' : fcEval, 'noDim' : graph['noCities'], 'noBits' : 8,'alpha':0.4}
    
        # store the best/average solution of each iteration (for a final plot used to anlyse the GA's convergence)
        bestRepres = []
        bestFitness = -999999
    
        ga = GA(gaParam, problParam)
        ga.initialisation()
        ga.evaluation(graph)
        
        for g in range(gaParam['noGen']):
            #plotting preparation
            bestSolX = ga.bestChromosome().repres
            bestSolY = ga.bestChromosome().fitness
            if bestSolY > bestFitness :
                bestFitness = bestSolY
                bestRepres.clear()
                for elem in bestSolX:
                    bestRepres.append(elem)
        
            #logic alg
            ga.oneGenerationSteadyState(graph)
            #ga.oneGenerationElitism(graph)
            # ga.oneGenerationSteadyState()
        
            bestChromo = ga.bestChromosome()
            print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness)) 
    
        print('\n\nBest solution is x = ' + str(bestRepres) + ' f(x) = ' + str(bestFitness)+' cost '+str(1/bestFitness))
            
        print('Best route:'+str(bestRepres))

        

In [117]:
import numpy as np 
import matplotlib.pyplot as plt 
import warnings 
def plotInitial(network):
    warnings.simplefilter('ignore')

    A=np.matrix(network["mat"])
    G=nx.from_numpy_matrix(A)
    pos = nx.spring_layout(G)  # compute graph layout
    plt.figure(figsize=(10, 10))  # image is 8 x 8 inches 
    nx.draw_networkx_nodes(G, pos, node_size=200, cmap=plt.cm.RdYlBu)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.show(G)


In [118]:
# plot a particular division in communities
def plotDivision(network,bestRepres):
    A=np.matrix(network["mat"])
    G=nx.from_numpy_matrix(A)
    pos = nx.spring_layout(G)  # compute graph layout
    plt.figure(figsize=(10, 10))  # image is 8 x 8 inches 
    nx.draw_networkx_nodes(G, pos, node_size = 200, cmap = plt.cm.RdYlBu, node_color = bestRepres)
    nx.draw_networkx_edges(G, pos, alpha = 0.3)
    plt.show(G)


In [119]:
from src.sapt5.UI import UI
solver=Solver()
UI=UI(solver)
UI.run()

Best solution in generation 0 is: x = [0, 1, 2, 4, 36, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 43, 42, 41, 44, 45, 46, 47, 48, 49, 50] f(x) = 0.0010553832555023096
Best solution in generation 1 is: x = [0, 1, 2, 3, 36, 4, 5, 6, 7, 8, 9, 10, 11, 13, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 39, 23, 24, 25, 26, 27, 28, 37, 30, 31, 32, 33, 34, 35, 29, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50] f(x) = 0.0010713739108887082
Best solution in generation 2 is: x = [0, 1, 2, 3, 36, 4, 5, 6, 7, 8, 9, 10, 11, 13, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 39, 23, 24, 25, 26, 27, 28, 37, 30, 31, 32, 33, 34, 35, 29, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50] f(x) = 0.0010713739108887082
Best solution in generation 3 is: x = [0, 3, 4, 5, 6, 50, 7, 41, 8, 9, 10, 11, 17, 13, 14, 15, 36, 16, 12, 18, 19, 20, 33, 29, 21, 22, 23, 24, 25, 37, 26, 27, 28, 30, 31, 32, 35, 34, 38, 39, 40, 42, 43, 44, 45, 46,